<a href="https://colab.research.google.com/github/sudeepjd/Data-Analytics/blob/master/09-Natural%20Language%20Processing/ML_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Machine Learning Chatbot using SVM Classifier

## Import the Libraries

In [ ]:
import nltk
import numpy as np
import json
import random
import urllib
from nltk.stem.lancaster import LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# nltk.download('punkt')

## Load the Intents File

In [ ]:
file = urllib.request.urlopen("https://raw.githubusercontent.com/sudeepjd/Data-Analytics/master/09-Natural%20Language%20Processing/mlchat_intents.json")
data = json.loads(file.read())

Extract the Data

In [ ]:
docs_x = []
docs_y = []

In [ ]:
stemmer = LancasterStemmer()
for intent in data['intents']:
	for pattern in intent['patterns']:
		wrds = nltk.word_tokenize(pattern)
		wrds = [stemmer.stem(w.lower()) for w in wrds if w != "?"]
		docs_x.append(' '.join(wrds))
		docs_y.append(intent["tag"])

Vectorize X

In [ ]:
cv = CountVectorizer()
X = cv.fit_transform(docs_x).toarray()
print(X)

Vectorize Y

In [ ]:
le = LabelEncoder()
y = le.fit_transform(docs_y)

## Build and Train the Chatbot

Support Vector Machine

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel='linear')
classifier.fit(X, y)

Predict Function

In [ ]:
def provide_response(sent):
  #Pre-Process Input
  sent = nltk.word_tokenize(sent)
  sent = [stemmer.stem(w.lower()) for w in sent if w != "?"]
  sent = [' '.join(sent)]
  X_pred = cv.transform(sent).toarray()

  #Predict
  y_pred = classifier.predict(X_pred)
  tag = le.inverse_transform(y_pred)
  tag = tag[0]

  #Get the options to respond
  for tg in data['intents']:
    if tg['tag'] == tag:
      responses = tg['responses']

  #Return one of them
  return (tag, random.choice(responses))

Test Single Response

In [ ]:
print(provide_response("What can I do with regression?"))

## Execute the Chatbot

In [ ]:
print ("BOT : Hello! I am ML Chat.")

while True:
  inp = input("\nYOU : ")
  resp = provide_response(inp)
  
  print("\nBOT : " + resp[1])
  
  if resp[0] == "goodbye" : break